In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
from huggingface_hub import login

login(token="your_token")

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/aeolian83/.cache/huggingface/token
Login successful


In [3]:
from datasets import load_dataset

data = load_dataset("aeolian83/DnD_translate")

In [4]:
data

DatasetDict({
    train: Dataset({
        features: ['num', 'english', 'korean'],
        num_rows: 96785
    })
})

In [5]:
type(data)

datasets.dataset_dict.DatasetDict

In [6]:
# 데이터 확인

print(data['train'][20000]['korean'])
print()
print(data['train'][20000]['english'])

레익 게스라스

Rayic Gethras


In [7]:
# data 형태를 prompt에 맞추기
# korean과 english로 나뉜 데이터를 text로 통합
# datasets의 map함수 사용법 https://huggingface.co/docs/datasets/v2.14.4/en/process#map

def makedata(x):
    return {'text': f"### 영어: {x['english']}</끝>\n### 한국어: {x['korean']}</끝>"}

data = data.map(makedata)

In [8]:
data['train'][100]

{'num': 100,
 'english': "Stop a second there, young 'un, 'cuz I heard what you gone and done for old Brun. It made me ashamed that I didn't try to help him myself. Good to know there's still someone willin' ta go the extra mile, even for a stranger. I want you to have this heirloom of mine because of what you did. Now my heirs didn't loom so big, but you ought to get more use out o' it than my mantle does. Folks like you make a jaded old man think there's still decent people out there.",
 'korean': '잠깐만 젊은이, 자네가 브룬을 여러모로 힘써주었다는 얘기를 들었다네. 난 그 녀석을 직접 도우려고 시도조차 하지 않았지, 부끄러울 따름이네. 아직까지 낯선 사람을 선뜻 도와주는 사람들이 남아있다는 게 기쁘군. 나는 그 보답으로 우리 집안의 가보를 주고 싶네. 별 볼 일 없는 가보라도, 나보다는 자네에게 더 쓸모 있을 것 같아. 자네 같은 친구 덕에 아직도 세상엔 착한 사람들이 남아있다는 걸 이 노인네도 실감했다네.',
 'text': "### 영어: Stop a second there, young 'un, 'cuz I heard what you gone and done for old Brun. It made me ashamed that I didn't try to help him myself. Good to know there's still someone willin' ta go the extra mile, even for a stranger. I want you to h

In [9]:
data['train'][50000]

{'num': 50000,
 'english': 'Good. Stay here. I must find Lady Galvena and, if the Lady is cooperative, Claire.',
 'korean': '좋아. 여기 머물러. 갈베나 여사와, 만약 그녀가 협조적이라면, 클레어를 반드시 찾겠어.',
 'text': '### 영어: Good. Stay here. I must find Lady Galvena and, if the Lady is cooperative, Claire.</끝>\n### 한국어: 좋아. 여기 머물러. 갈베나 여사와, 만약 그녀가 협조적이라면, 클레어를 반드시 찾겠어.</끝>'}

In [10]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

model_id = "squarelike/Gugugo-koen-1.3B-V1.0"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

In [11]:
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

In [12]:
data

DatasetDict({
    train: Dataset({
        features: ['num', 'english', 'korean', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 96785
    })
})

In [13]:
data["train"][20000]["text"]

'### 영어: Rayic Gethras</끝>\n### 한국어: 레익 게스라스</끝>'

In [14]:
data["train"][90000]

{'num': 90000,
 'english': 'And why not? We are a force to be reckoned with, a power matched by few. ',
 'korean': '그러면 안 될 것도 있나? 우리는 누구도 무시할 수 없는 사람이다, 우리 힘에 대적할 자는 거의 없다.',
 'text': '### 영어: And why not? We are a force to be reckoned with, a power matched by few. </끝>\n### 한국어: 그러면 안 될 것도 있나? 우리는 누구도 무시할 수 없는 사람이다, 우리 힘에 대적할 자는 거의 없다.</끝>',
 'input_ids': [6,
  6,
  6,
  3029,
  29,
  1300,
  17130,
  9883,
  23153,
  10590,
  7908,
  34,
  3792,
  72,
  224,
  17081,
  12063,
  16092,
  9138,
  13384,
  8517,
  72,
  21924,
  70,
  78,
  2600,
  6057,
  25614,
  15,
  12063,
  8669,
  28559,
  7789,
  3432,
  29537,
  71,
  8517,
  92,
  8759,
  11594,
  17,
  422,
  18,
  5568,
  33,
  202,
  6,
  6,
  6,
  8611,
  29,
  4275,
  596,
  1180,
  388,
  309,
  327,
  392,
  34,
  848,
  272,
  2041,
  309,
  4527,
  482,
  365,
  550,
  272,
  790,
  270,
  267,
  15,
  848,
  1156,
  274,
  351,
  407,
  482,
  454,
  272,
  2440,
  550,
  267,
  17,
  31,
  18,
  5568,
  33],
 'toke

In [15]:
data["train"][20000]["english"]

'Rayic Gethras'

In [16]:
print(tokenizer.decode(1598))

 레


In [17]:
tokenizer.encode(data["train"][20000]["text"])

[6,
 6,
 6,
 3029,
 29,
 3069,
 7743,
 4261,
 2020,
 5145,
 75,
 85,
 7245,
 31,
 18,
 5568,
 33,
 202,
 6,
 6,
 6,
 8611,
 29,
 1598,
 1102,
 904,
 387,
 3893,
 31,
 18,
 5568,
 33]

In [18]:
print(tokenizer.decode(3029))
print(tokenizer.decode(8611))
print(tokenizer.decode(29))

 영어
 한국어
:


In [19]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [20]:
def print_trainable_parameters(model):
    """
    Print the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainble%: {100 * trainable_params / all_param}"
    )

In [21]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)


trainable params: 1572864 || all params: 729403392 || trainble%: 0.21563705588032142


In [22]:
tokenNum_korean = 8611
tokenNum_english = 3029
tokenNum_colon = 29

In [23]:
import transformers
from transformers import Trainer
import numpy as np

class maskTrainer(Trainer):
  def __init__(self, *args, **kwargs):
    super().__init__(*args, **kwargs)

  def compute_loss(self, model, inputs, return_outputs=False):
    # print(inputs['labels'][1])
    for x in range(len(inputs['labels'])):
      if (inputs['labels'][x][3] == tokenNum_korean):
          maskindex = (inputs['labels'][x]==tokenNum_english).nonzero()[:, 0]
          temp = 0
          for i, index in enumerate(maskindex):
            if (inputs['labels'][x][index+1] != tokenNum_colon):
              maskindex = np.delete(maskindex.cpu(), i-temp)
              temp += 1

      elif (inputs['labels'][x][3] == tokenNum_english):
          maskindex = (inputs['labels'][x]==tokenNum_korean).nonzero()[:, 0]
          temp = 0
          for i, index in enumerate(maskindex):
            if (inputs['labels'][x][index+1] != tokenNum_colon):
              maskindex = np.delete(maskindex.cpu(), i-temp)
              temp += 1

      inputs['labels'][x][:maskindex[0]+2] = -100

    # print(inputs['labels'][1])
    outputs = model(**inputs)

    loss = outputs['loss']

    return (loss,outputs) if return_outputs else loss

In [24]:
# import transformers

tokenizer.pad_token = tokenizer.eos_token

trainer = maskTrainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=1,
        fp16=True,
        output_dir="outputs",
        save_total_limit=5,
        logging_steps=300,
        report_to=["tensorboard"],
        num_train_epochs = 3,
        learning_rate=3e-4,
        resume_from_checkpoint=True,
        lr_scheduler_type= "cosine",
        #optim="paged_adamw_8bit"

    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train(resume_from_checkpoint=False)

/home/aeolian83/anaconda3/envs/translateDnD/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
300,1.529900
600,1.472600
900,1.423300
1200,1.459200
1500,1.482000
1800,1.439200
2100,1.450500
2400,1.445000
2700,1.404900
3000,1.360700


TrainOutput(global_step=290355, training_loss=1.156305467252473, metrics={'train_runtime': 33937.8302, 'train_samples_per_second': 8.555, 'train_steps_per_second': 8.555, 'total_flos': 1.1054036567089152e+17, 'train_loss': 1.156305467252473, 'epoch': 3.0})

In [25]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [26]:
model.save_pretrained("./qlora/translation/Gugugo_for_DnD_v/")

In [27]:
# from transformers import StoppingCriteria, StoppingCriteriaList

# class StoppingCriteriaSub(StoppingCriteria):
#     def __init__(self, stops = [], encounters=1):
#         super().__init__()
#         self.stops = [stop for stop in stops]

#     def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
#         for stop in self.stops:
#             if torch.all((stop == input_ids[0][-len[stop]:])).item():
#                 return True
            
#         return False
    
# stop_words = ["</끝>"]
# stop_words_ids = [tokenizer(stop_word, return_tensors='pt')['input_ids'].squeeze() for stop_word in stop_words]
# stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops = stop_words_ids)])


In [28]:
from transformers import StoppingCriteria, StoppingCriteriaList

class StoppingCriteriaSub(StoppingCriteria):

    def __init__(self, stops = [], encounters=1):
        super().__init__()
        self.stops = [stop for stop in stops]

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        for stop in self.stops:
            if torch.all((stop == input_ids[0][-len(stop):])).item():
                return True

        return False

stop_words = ["</끝>"]
stop_words_ids = [tokenizer(stop_word, return_tensors='pt')['input_ids'].squeeze() for stop_word in stop_words]
stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words_ids)])

In [29]:
def gen(lan="en", x=""):
    if (lan == "ko"):
        prompt = f"### 한국어: {x}</끝>\n### 영어:"
    else:
        prompt = f"### 영어: {x}</끝>\n### 한국어:"
    gened = model.generate(
        **tokenizer(
            prompt,
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=100,
        temperature=0.001,
        no_repeat_ngram_size=10,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
        stopping_criteria=stopping_criteria
    )
    return tokenizer.decode(gened[0]).replace(prompt+" ", "")

In [36]:
gen(lan="en", x="i am a student")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'나는 학생입니다</끝>'

In [31]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(30080, 2048)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-23): 24 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXAttention(
              (rotary_emb): GPTNeoXRotaryEmbedding()
              (query_key_value): Linear4bit(
                in_features=2048, out_features=6144, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=

In [35]:
gen(lan="en", x="You remind me of that mouthy gnome.")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'그 입이 험한 노움을 연상시키는군.</끝>'

In [43]:
trainer = maskTrainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=1,
        fp16=True,
        output_dir="outputs",
        save_total_limit=5,
        logging_steps=200,
        report_to=["tensorboard"],
        num_train_epochs = 6,
        learning_rate=3e-5,
        resume_from_checkpoint=True,
        lr_scheduler_type= "cosine",
        #optim="paged_adamw_8bit"

    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train(resume_from_checkpoint=True)

/home/aeolian83/anaconda3/envs/translateDnD/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
389600,0.911500
389800,0.951000
390000,0.978800
390200,0.992800
390400,0.950100
390600,0.997000
390800,0.887900
391000,0.947100
391200,1.011200
391400,0.963800


TrainOutput(global_step=580710, training_loss=0.3034457198761134, metrics={'train_runtime': 22445.9205, 'train_samples_per_second': 25.872, 'train_steps_per_second': 25.872, 'total_flos': 2.2108073134178304e+17, 'train_loss': 0.3034457198761134, 'epoch': 6.0})

In [44]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference

In [45]:
model.save_pretrained("./qlora/translation/Gugugo_for_DnD_v0.8/")